# Using the pluto MutualFundsIndia data-set

In [1]:
import pandas as pd
from sqlalchemy import func, or_, and_
import plutoPy
from plutoPy.model import RoutingSession, MutualFundsIndia

### who are the biggest 5 mutual fund managers?

In [2]:
lastAumDate = RoutingSession.session.query(func.max(MutualFundsIndia.AumFundwise.PERIOD)).scalar()
print(f"{lastAumDate}")

aums = (RoutingSession.session.query(MutualFundsIndia.AumFundwise.FUND, MutualFundsIndia.AumFundwise.AVG_AUM_WO_FOFD + MutualFundsIndia.AumFundwise.AVG_AUM_FOFD)
        .filter(MutualFundsIndia.AumFundwise.PERIOD == lastAumDate)
        .order_by((MutualFundsIndia.AumFundwise.AVG_AUM_WO_FOFD + MutualFundsIndia.AumFundwise.AVG_AUM_FOFD).desc())).all()

aumDf = pd.DataFrame(aums, columns=['FUND', 'AUM']) #analyze this!
print(aumDf[0:5])

2019-06-01
                                FUND          AUM
0                   HDFC Mutual Fund  36276279.17
1       ICICI Prudential Mutual Fund  33876819.92
2                    SBI Mutual Fund  30784117.01
3  Aditya Birla Sun Life Mutual Fund  25418197.61
4               Reliance Mutual Fund  22327193.29


/usr/local/lib/python3.6/dist-packages/sqlalchemy/dialects/mssql/base.py:2307: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sys.dm_pdw_nodes_exec_sessions'.")
  "tried views: %s; final error was: %s" % (views, last_error)


### what are the largest 10 mid-cap funds by AUM?

In [3]:
# Meta to filter for the 'Mid-Cap' category
# AumSchemewise to sort funds by AUM - largest first
# NavTimeSeries to get the start and end dates for which NAVs are available

lastMetaDate = RoutingSession.session.query(func.max(MutualFundsIndia.Meta.AS_OF)).scalar()
lastSwDate = RoutingSession.session.query(func.max(MutualFundsIndia.AumSchemewise.PERIOD)).scalar()
print(f"{lastMetaDate}/{lastSwDate}")

2019-06-22/2019-06-01


In [4]:
midCaps = (RoutingSession.session.query(MutualFundsIndia.Meta.SCHEME_CODE, 
                                       MutualFundsIndia.AumSchemewise.SCHEME_NAME,
                                       MutualFundsIndia.Meta.CATEGORY, 
                                       MutualFundsIndia.AumSchemewise.AVG_AUM_WO_FOFD,
                                       func.min(MutualFundsIndia.NavTimeSeries.TIME_STAMP).label("start_dt"), 
                                       func.max(MutualFundsIndia.NavTimeSeries.TIME_STAMP).label("end_dt"))
        .join(MutualFundsIndia.AumSchemewise, MutualFundsIndia.Meta.SCHEME_CODE == MutualFundsIndia.AumSchemewise.SCHEME_CODE)
        .join(MutualFundsIndia.NavTimeSeries, MutualFundsIndia.NavTimeSeries.SCHEME_CODE == MutualFundsIndia.AumSchemewise.SCHEME_CODE)
        .filter(and_(MutualFundsIndia.AumSchemewise.PERIOD == lastSwDate,
                    MutualFundsIndia.Meta.AS_OF == lastMetaDate, 
                    MutualFundsIndia.Meta.CATEGORY == 'Mid-Cap'))
        .group_by(MutualFundsIndia.Meta.SCHEME_CODE, 
            MutualFundsIndia.AumSchemewise.SCHEME_NAME,
            MutualFundsIndia.Meta.CATEGORY, 
            MutualFundsIndia.AumSchemewise.AVG_AUM_WO_FOFD)
        .order_by(MutualFundsIndia.AumSchemewise.AVG_AUM_WO_FOFD.desc())).all()

print(f"fetched: {len(midCaps)}")

# get a pandas dataframe out of the list of tuples
mcDf = pd.DataFrame(midCaps)

pd.set_option('display.max_columns', None)
print(mcDf[0:10])

fetched: 15
   SCHEME_CODE                                        SCHEME_NAME CATEGORY  \
0       104481           DSP  Midcap Fund - Regular Plan - Growth  Mid-Cap   
1       100473                   Franklin India Prima Fund-Growth  Mid-Cap   
2       101539                      Sundaram Mid Cap Fund -Growth  Mid-Cap   
3       112496           L&amp;T Mid Cap Fund-Regular Plan-Growth  Mid-Cap   
4       118533        Franklin India Prima Fund - Direct - Growth  Mid-Cap   
5       119807            L&amp;T Mid Cap Fund-Direct Plan-Growth  Mid-Cap   
6       119071            DSP  Midcap Fund - Direct Plan - Growth  Mid-Cap   
7       119581  Sundaram Mid Cap Fund- Direct Plan - Growth Op...  Mid-Cap   
8       105503          Invesco India Midcap Fund - Growth Option  Mid-Cap   
9       120403  Invesco India Midcap Fund - Direct Plan - Grow...  Mid-Cap   

   AVG_AUM_WO_FOFD    start_dt      end_dt  
0        486411.17  2006-11-16  2019-07-01  
1        475443.94  2006-04-03  2019-07

### get the NAV time-series

In [5]:
scode = int(mcDf.iloc[2]['SCHEME_CODE'])
navs = (RoutingSession.session.query(MutualFundsIndia.NavTimeSeries.TIME_STAMP, MutualFundsIndia.NavTimeSeries.NAV)
        .filter(MutualFundsIndia.NavTimeSeries.SCHEME_CODE == scode)
        .order_by(MutualFundsIndia.NavTimeSeries.TIME_STAMP)).all()

navDf = pd.DataFrame(navs) #analyze this!
print(navDf[0:10])         

   TIME_STAMP      NAV
0  2006-04-03  77.4753
1  2006-04-04  78.3241
2  2006-04-05  80.2362
3  2006-04-07  79.9437
4  2006-04-10  80.6520
5  2006-04-12  79.5576
6  2006-04-13  78.0991
7  2006-04-17  79.1493
8  2006-04-18  80.0763
9  2006-04-19  80.7010
